In [ ]:
!pip install -U pathway
!pip install -U bokeh
 # Installing pathway

In [ ]:
#importing all important libraries
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn
import matplotlib.pyplot as plt
from datetime import timedelta
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()


In [ ]:
df = pd.read_csv('/content/dataset.csv')
df

In [ ]:
df['QueueLength'].max()

In [ ]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

# Sort the DataFrame by the new 'Timestamp' column and reset the index
df = df.sort_values('Timestamp').reset_index(drop=True)

In [ ]:
#encoding vehicle type ( bike : 0.7, car : 1, truck:1.5)
vehicle_type_mapping = {'bike':0.7, 'car': 1, 'truck': 1.5,'cycle':0.5}
df['VehicleTypeEncoded'] = df['VehicleType'].map(vehicle_type_mapping).fillna(-1).astype(float)
#encoding traffic condition
traffic_condition_mapping = {'low': -0.5, 'average': 1, 'high': 1.5}
df['TrafficConditionEncoded'] = df['TrafficConditionNearby'].map(traffic_condition_mapping).fillna(-1).astype(float)
display(df.tail())

MODEL 1 - BASELINE LINEAR MODEL

In [ ]:
import os
#create a file with only necessary columns i.e. occupancy, timestamp and capacity
# Create output directory
os.makedirs("garages", exist_ok=True)

# Group by unique garage locations (Lat, Long)
for (lat, lon), group in df.groupby(["Latitude", "Longitude"]):
    filename = f"garages/garage_{lat:.5f}_{lon:.5f}.csv"
    group.to_csv(filename, index=False)

print("✅ Done splitting into garage-wise files.")

In [ ]:
from datetime import timedelta
import math
import os
import pathway as pw
import panel as pn
import bokeh.plotting

# Enable Panel
pn.extension()

# List of all garage files
garage_files = os.listdir("garages")

# Container for all visualizations
visualizations = []

# Loop over each garage CSV
for file in sorted(garage_files):
    path = f"garages/{file}"
    print(f"▶️ Running for: {file}")

    class Schema(pw.Schema):
        Timestamp: str
        Occupancy: int
        Capacity: int

    data = pw.demo.replay_csv(path, schema=Schema, input_rate=100)

    # Parse time and extract date
    format = "%Y-%m-%d %H:%M:%S"
    data_with_time = data.with_columns(
        time = data.Timestamp.dt.strptime(format),
        date = data.Timestamp.dt.strptime(format).dt.strftime("%Y-%m-%dT00:00:00")
    )

    # Rolling window calculations
    window = (
        data_with_time.windowby(
            pw.this.time,
            instance=pw.this.date,
            window=pw.temporal.tumbling(timedelta(minutes=30)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t=pw.this._pw_window_end,
            avg_occ=pw.reducers.avg(pw.this.Occupancy),
            current_occ = pw.reducers.max(pw.this.Occupancy),
            cap=pw.reducers.max(pw.this.Capacity),
        )
    )

    # Shift previous window for lag comparison
    prev_window = window.with_columns(t=pw.this.t + timedelta(minutes=30))

    # Join current and previous window
    joined = window.join(
        prev_window,
        pw.this.t == pw.right.t,
        how=pw.JoinMode.LEFT
    ).select(
        window_end = pw.this.t,
        avg_occ_now = pw.left.avg_occ,
        avg_occ_prev = pw.right.avg_occ,
        cap = pw.left.cap,
        current_occ = pw.left.current_occ
    )

    # Fill missing previous occupancy with current
    joined = joined.with_columns(
        avg_occ_prev=pw.if_else(
            pw.this.avg_occ_prev.is_none(),
            pw.this.avg_occ_now,
            pw.this.avg_occ_prev
        )
    )

    # Compute delta and load factor
    joined = joined.with_columns(
        delta_occ=(pw.this.avg_occ_now - pw.this.avg_occ_prev) / pw.this.cap,
        load_factor = pw.this.current_occ / pw.this.cap
    )

    # Final price
    alpha = 10
    beta = 4.0
    base_price = 10
    final_price_table = joined.with_columns(
        price=base_price + alpha * pw.this.delta_occ + beta * pw.this.load_factor
    ).rename_columns(timestam = "window_end")

    # Plotting function
    def price_plotter(source):
        fig = bokeh.plotting.figure(
            height=350,
            width=700,
            title=f"Dynamic Pricing - {file}",
            x_axis_type="datetime",
        )
        fig.line("timestam", "price", source=source, line_width=2, color="green")
        fig.circle("timestam", "price", source=source, size=6, color="red")
        return fig

    # Append graph
    viz = final_price_table.plot(price_plotter, sorting_col="timestam")
    visualizations.append(viz)

# Layout all garage graphs in a column
pn.Column(*visualizations).servable()

In [ ]:
pw.run()

MODEL 2

In [ ]:
from datetime import timedelta
import pathway as pw
import panel as pn
import bokeh.plotting
import math
import os

# Setup Panel extension
pn.extension()

# Constants
max_queue = 15
max_traffic = 2
base_price = 10.0

# Plot container
panels = []

# Loop over each garage file
garage_dir = "garages"
garage_files = os.listdir(garage_dir)

for file in sorted(garage_files):
    path = f"{garage_dir}/{file}"
    print(f"▶️ Running for: {file}")

    class Schema(pw.Schema):
        Timestamp: str
        Occupancy: int
        Capacity: int
        IsSpecialDay: int
        TrafficConditionEncoded: float
        VehicleTypeEncoded: float
        QueueLength: int

    data = pw.demo.replay_csv(path, schema=Schema, input_rate=100)

    format = "%Y-%m-%d %H:%M:%S"
    data_with_time = data.with_columns(
        time = data.Timestamp.dt.strptime(format),
        date = data.Timestamp.dt.strptime(format).dt.strftime("%Y-%m-%dT00:00:00")
    )
#define window
    window = (
        data_with_time.windowby(
            pw.this.time,
            instance=pw.this.date,
            window=pw.temporal.tumbling(timedelta(minutes = 30)),
            behavior=pw.temporal.exactly_once_behavior()
        )
        .reduce(
            t = pw.this._pw_window_end,
            avg_occ = pw.reducers.avg(pw.this.Occupancy),
            current_occ = pw.reducers.max(pw.this.Occupancy),
            cap = pw.reducers.max(pw.this.Capacity),
            is_special_day = pw.reducers.max(pw.this.IsSpecialDay),
            traffic_condition = pw.reducers.max(pw.this.TrafficConditionEncoded),
            VehicleTypeEncoded = pw.reducers.max(pw.this.VehicleTypeEncoded),
            queue_length = pw.reducers.max(pw.this.QueueLength)
        )
    )
#for savin previous occupancy
    prev_window = window.with_columns(t = pw.this.t + timedelta(minutes=30))

    joined = window.join(
        prev_window,
        pw.this.t == pw.right.t,
        how=pw.JoinMode.LEFT
    ).select(
        window_end = pw.this.t,
        avg_occ_now = pw.left.avg_occ,
        cap = pw.left.cap,
        current_occ = pw.left.current_occ,
        prev_occupancy = pw.right.avg_occ,
        is_special_day = pw.left.is_special_day,
        traffic_condition = pw.left.traffic_condition,
        VehicleTypeEncoded = pw.left.VehicleTypeEncoded,
        queue_length = pw.left.queue_length
    )

    joined = joined.with_columns(
        prev_occupancy_clean = pw.if_else(
            pw.this.prev_occupancy.is_none(),
            pw.this.current_occ,
            pw.this.prev_occupancy
        )
    )

    joined = joined.with_columns(
        delta = (pw.this.current_occ - pw.this.prev_occupancy_clean) / pw.this.cap,
        occupancy_rate = pw.this.current_occ / pw.this.cap
    )
#feature engineering
    joined = joined.with_columns(
        f_occ = pw.apply(lambda x: 1 / (1 + math.exp(-10 * (x - 1))), pw.this.occupancy_rate),
        f_queue_norm = pw.apply(lambda x: math.log1p(x) / math.log1p(max_queue), pw.this.queue_length),
        f_traffic = pw.apply(lambda x: math.log1p(x) / math.log1p(max_traffic), pw.this.traffic_condition)
    )

#defining demand
    joined = joined.with_columns(
        demand_factor = pw.apply(
            lambda f_occ, f_queue, f_traffic, delta, is_special_day, vehicle_weight : (
                (3 * f_occ + 1.5 * f_queue + 1.2 * f_traffic + 1.3 * delta) *
                (is_special_day + 1) * vehicle_weight
            ),
            pw.this.f_occ,
            pw.this.f_queue_norm,
            pw.this.f_traffic,
            pw.this.delta,
            pw.this.is_special_day,
            pw.this.VehicleTypeEncoded
        )
    )
    joined = joined.with_columns(
        price = pw.apply(
            lambda demand: base_price * (1 + 0.06 * demand),
            pw.this.demand_factor
        )
    )
#defining the pricing function
    final_table = joined.select(
        window_end = pw.this.window_end,
        price = pw.this.price
    ).rename_columns(timestam="window_end")
#plotting using bokeh
    def price_plotter(source):
        fig = bokeh.plotting.figure(
            height=300,
            width=800,
            title=f"Dynamic Price: {file}",
            x_axis_type="datetime"
        )
        fig.line("timestam", "price", source=source, line_width=2, color="navy")
        fig.circle("timestam", "price", source=source, size=5, color="red")
        return fig

    panels.append(final_table.plot(price_plotter, sorting_col="timestam"))

# Show all graphs in a scrollable column
pn.Column(*panels).servable()




In [ ]:
#run
pw.run()

Defining my demand => It will be based on 5 factors
I will design a demand function and normalize it.